In [1]:
import asyncio
import pandas as pd
from coworker.modules.coworker import Coworker

coworker = Coworker()

In [2]:
def print_results(info, summary, query, data):
    if info:
        print("Here is some info to begin with:\n")
        print(pd.DataFrame(info))
        print('')
    print(summary)
    print('')
    print(query)
    print('')
    if data != 'Apologies. I cannot figure out a way to answer this request.':
        print(pd.DataFrame(data).head())
    else:
        print(data)

In [3]:
question = '''
What were the net liabilities for apple and microsoft in 2018?
'''

# Create tasks
task_get_prelim_info = asyncio.create_task(coworker.get_info_from_text(question))
task_summary = asyncio.create_task(coworker.generate_summary(question))
task_query_and_data = asyncio.create_task(coworker.process_query_and_data(question))

# Await tasks concurrently
info, summary, (query, data) = await asyncio.gather(task_get_prelim_info, task_summary, task_query_and_data)

print_results(info, summary, query, data)

Here is some info to begin with:

   Code                   Name Exchange CurrencyCode      Sector  \
0  AAPL              Apple Inc   NASDAQ          USD  Technology   
1  MSFT  Microsoft Corporation   NASDAQ          USD  Technology   

                  Industry                                        Description  \
0     Consumer Electronics  Apple Inc. designs, manufactures, and markets ...   
1  Software-Infrastructure  Microsoft Corporation develops, licenses, and ...   

                                                 CEO     IPODate  \
0  {'Name': 'Mr. Timothy D. Cook', 'Title': 'CEO ...  1980-12-12   
1  {'Name': 'Mr. Satya  Nadella', 'Title': 'Chair...  1986-03-13   

         Country       City  
0  United States  Cupertino  
1  United States    Redmond  

To determine the net liabilities for Apple and Microsoft in 2018, I would analyze their respective balance sheets from that year, specifically focusing on the difference between total liabilities and total assets.

SELECT

In [123]:
from eod import EodHistoricalData
from coworker.config.config import EOD_KEY
import json
import re

client = EodHistoricalData(EOD_KEY)

with open('coworker/data/companies.json', 'r') as f:
    companies_dict = json.load(f)

In [152]:
def get_info_from_text(text):
    text = re.sub('[^A-Za-z0-9& ]+', '', text)  # remove special characters
    words_company = [word for word in text.split(' ') if len(word) >= 4]
    words_ticker = [word for word in text.split(' ') if len(word) >= 2]
    
    tickers = []
    for word in words_company:
        for company, ticker in companies_dict.items():
            if word.lower() in company.lower():
                if ticker not in tickers:  # to avoid duplicate entries
                    tickers.append(ticker)
    
    for word in words_ticker:
        for company, ticker in companies_dict.items():
            if word == ticker:
                if ticker not in tickers:  # to avoid duplicate entries
                    tickers.append(ticker)

    info_dict = {
        'Code': [],
        'Name': [],
        'Exchange': [],
        'CurrencyCode': [],
        'Sector': [],
        'Industry': [],
        'Description': [],
        'CEO': [],
        'IPODate': [],
        'Country': [],
        'City': []
    }

    for ticker in tickers:
        info = client.get_fundamental_equity(ticker, filter_='General')
        
        info_dict['Code'].append(info['Code'])
        info_dict['Name'].append(info['Name'])
        info_dict['Exchange'].append(info['Exchange'])
        info_dict['CurrencyCode'].append(info['CurrencyCode'])
        info_dict['Sector'].append(info['Sector'])
        info_dict['Industry'].append(info['Industry'])
        info_dict['Description'].append(info['Description'])
        info_dict['CEO'].append(info['Officers']['0'])
        info_dict['IPODate'].append(info['IPODate'])
        info_dict['Country'].append(info['AddressData']['Country'])
        info_dict['City'].append(info['AddressData']['City'])

    return info_dict

In [151]:
text = "I would like to see assets and liabilities for Apple, Amazon! and Microsoft and Zoom - Blizzard, ASML, ABT, at&t! ACCENTURE, Costco"
get_info_from_text(text)

{'Code': ['AAPL',
  'AMZN',
  'MSFT',
  'ZM',
  'ATVI',
  'ASML',
  'T',
  'ACN',
  'COST',
  'ABT'],
 'Name': ['Apple Inc',
  'Amazon.com Inc',
  'Microsoft Corporation',
  'Zoom Video Communications Inc',
  'Activision Blizzard Inc',
  'ASML Holding NV ADR',
  'AT&T Inc',
  'Accenture plc',
  'Costco Wholesale Corp',
  'Abbott Laboratories'],
 'Exchange': ['NASDAQ',
  'NASDAQ',
  'NASDAQ',
  'NASDAQ',
  'NASDAQ',
  'NASDAQ',
  'NYSE',
  'NYSE',
  'NASDAQ',
  'NYSE'],
 'CurrencyCode': ['USD',
  'USD',
  'USD',
  'USD',
  'USD',
  'USD',
  'USD',
  'USD',
  'USD',
  'USD'],
 'Sector': ['Technology',
  'Consumer Cyclical',
  'Technology',
  'Technology',
  'Communication Services',
  'Technology',
  'Communication Services',
  'Technology',
  'Consumer Defensive',
  'Healthcare'],
 'Industry': ['Consumer Electronics',
  'Internet Retail',
  'Software-Infrastructure',
  'Software-Application',
  'Electronic Gaming & Multimedia',
  'Semiconductor Equipment & Materials',
  'Telecom Service